In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
null = df.isnull().sum()
max(null)

In [ ]:
#no null or missing values 

In [ ]:
labels_train = df['target']
df = df.drop(columns = ['id'])

In [ ]:
df.drop(columns = ['target'],inplace=True)

In [ ]:
unique = []
for i in df.columns:
    unique.append(df[i].unique().size)

In [ ]:
max(unique)

In [ ]:
# In 200000 data points only 105 different values max, All labels are categorical

In [ ]:
labels_train.unique()

In [ ]:
for i in df.columns:
    plt.figure()
    sns.distplot(df[i])
    plt.title(i)
    plt.show()

In [ ]:
#Every feature is right skewed

In [ ]:
labels_train.value_counts()

In [ ]:
plt.figure(figsize=(18,25))
sns.boxplot(data=df, orient="h")
plt.title('Outliers in Train Data')

In [ ]:
# sns.pairplot(df)

In [ ]:
map = {'Class_6':5, 'Class_2':1, 'Class_8':7, 'Class_3':2, 'Class_1':0, 'Class_5':4,
       'Class_7':6, 'Class_9':8, 'Class_4':3}
labels = labels_train.map(map)


In [ ]:
labels = np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split
train,val,labels_train,val_labels = train_test_split(df,labels,test_size = 0.3)

In [ ]:
labels_train

In [ ]:
labels_train = np.array(labels_train)

In [ ]:
test = test.drop(columns = ['id'])

In [ ]:
lab = []
for i in labels_train:
    lab.append(float(i))

In [ ]:
lab = np.array(lab)

In [ ]:
val_lab = []
for i in val_labels:
    val_lab.append(float(i))

In [ ]:
lab = pd.get_dummies(lab)
val_lab = pd.get_dummies(lab)

In [ ]:
lab.head()

In [ ]:
tf.keras.backend.clear_session()
# optimizer = tf.keras.optimizers.SGD(lr = 1e-6,momentum = 0.9)
model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(input_shape = (75,)),
    tf.keras.layers.Dense(72,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(36,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(30,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(150,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32,activation = 'relu'),
#     tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(9,activation = 'softmax')
])
early_stop = tf.keras.callbacks.EarlyStopping(patience = 50)
model_check = tf.keras.callbacks.ModelCheckpoint('model.h5',save_best_only=True)
model.compile(loss = 'sparse_categorical_crossentropy',optimizer ='adam',metrics = ['accuracy'])
history = model.fit(train,labels_train,epochs=600,callbacks = [model_check,early_stop],batch_size=64,validation_data = (val,val_labels))

In [ ]:
plt.figure()
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend()
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
tf.keras.backend.clear_session()
# optimizer = tf.keras.optimizers.SGD(lr = 1e-6,momentum = 0.9)
model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(input_shape = (75,)),
    tf.keras.layers.Dense(72,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(36,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(30,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(150,activation = 'relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32,activation = 'relu'),
#     tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(9,activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy',optimizer ='adam',metrics = ['accuracy'])
history = model.fit(df,labels,epochs=10,batch_size=64)

In [ ]:
# plt.figure()
# plt.plot(history.history['val_loss'])
# plt.plot(history.history['loss'])
# plt.legend()
# plt.ylabel('Loss')
# plt.xlabel('epoch')
# plt.show()

In [ ]:
pred = model.predict_proba(test)

In [ ]:
# pred = model.predict_proba(test)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
pred.head()

In [ ]:
pred.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']

In [ ]:
id = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
id = id['id']
pred['id'] = id

In [ ]:
pred.head()

In [ ]:
pred.to_csv('submit_3.csv',index = False)